# MNIST Analysis with Distributed Keras

**Joeri Hermans** (Technical Student, IT-DB-SAS, CERN)             
*Departement of Knowledge Engineering*         
*Maastricht University, The Netherlands*

In [1]:
!(date +%d\ %B\ %G)

18 January 2017


In this notebook we will show you how to process the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset using Distributed Keras. As in the [workflow](https://github.com/JoeriHermans/dist-keras/blob/master/examples/workflow.ipynb) notebook, we will guide you through the complete machine learning pipeline.

## Preparation

To get started, we first load all the required imports. Please make sure you installed `dist-keras`, and `seaborn`. Furthermore, we assume that you have access to an installation which provides Apache Spark.

Before you start this notebook, place make sure you ran the "MNIST preprocessing" notebook first, since we will be evaluating a manually "enlarged dataset".

In [2]:
%matplotlib inline

import numpy as np

from keras.optimizers import *
from keras.models import Sequential
from keras.layers.core import *
from keras.layers.convolutional import *

from pyspark import SparkContext
from pyspark import SparkConf

from matplotlib import pyplot as plt

from pyspark import StorageLevel

from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from distkeras.trainers import *
from distkeras.predictors import *
from distkeras.transformers import *
from distkeras.evaluators import *
from distkeras.utils import *

Using TensorFlow backend.


In the following cell, adapt the parameters to fit your personal requirements.

In [3]:
# Modify these variables according to your needs.
application_name = "Distributed Keras MNIST Analysis"
using_spark_2 = False
local = False
path = "mnist.parquet"
if local:
    # Tell master to use local resources.
    master = "local[*]"
    num_processes = 3
    num_executors = 1
else:
    # Tell master to use YARN.
    master = "yarn-client"
    num_executors = 30
    num_processes = 1

In [4]:
# This variable is derived from the number of cores and executors, and will be used to assign the number of model trainers.
num_workers = num_executors * num_processes

print("Number of desired executors: " + `num_executors`)
print("Number of desired processes / executor: " + `num_processes`)
print("Total number of workers: " + `num_workers`)

Number of desired executors: 30
Number of desired processes / executor: 1
Total number of workers: 30


In [5]:
conf = SparkConf()
conf.set("spark.app.name", application_name)
conf.set("spark.master", master)
conf.set("spark.executor.cores", `num_processes`)
conf.set("spark.executor.instances", `num_executors`)
conf.set("spark.locality.wait", "0")
conf.set("spark.executor.memory", "5g")
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer");

# Check if the user is running Spark 2.0 +
if using_spark_2:
    sc = SparkSession.builder.config(conf=conf) \
            .appName(application_name) \
            .getOrCreate()
else:
    # Create the Spark context.
    sc = SparkContext(conf=conf)
    # Add the missing imports
    from pyspark import SQLContext
    sqlContext = SQLContext(sc)

In [6]:
# Check if we are using Spark 2.0
if using_spark_2:
    reader = sc
else:
    reader = sqlContext
# Read the training and test set.
training_set = reader.read.parquet('data/mnist_train_big.parquet') \
                     .select("features_normalized_dense", "label_encoded", "label")
test_set = reader.read.parquet('data/mnist_test_preprocessed.parquet') \
                 .select("features_normalized_dense", "label_encoded", "label")

In [7]:
# Print the schema of the dataset.
training_set.printSchema()

root
 |-- features_normalized_dense: vector (nullable = true)
 |-- label_encoded: vector (nullable = true)
 |-- label: long (nullable = true)



## Model Development

### Multilayer Perceptron

In [8]:
mlp = Sequential()
mlp.add(Dense(1000, input_shape=(784,)))
mlp.add(Activation('relu'))
mlp.add(Dropout(0.2))
mlp.add(Dense(200))
mlp.add(Activation('relu'))
mlp.add(Dropout(0.2))
mlp.add(Dense(10))
mlp.add(Activation('softmax'))

In [9]:
mlp.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_1 (Dense)                  (None, 1000)          785000      dense_input_1[0][0]              
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 1000)          0           dense_1[0][0]                    
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 1000)          0           activation_1[0][0]               
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 200)           200200      dropout_1[0][0]                  
___________________________________________________________________________________________

In [10]:
optimizer_mlp = 'adam'
loss_mlp = 'categorical_crossentropy'

## Training

Prepare the training and test set for evaluation and training.

In [11]:
training_set = training_set.repartition(num_workers)
test_set = test_set.repartition(num_workers)
training_set.cache()
test_set.cache()
print("Number of training instances: " + str(training_set.count()))
print("Number of testing instances: " + str(test_set.count()))

Number of training instances: 6060000
Number of testing instances: 10000


## Evaluation

We define a utility function which will compute the accuracy for us.

In [12]:
def evaluate_accuracy(model, test_set, features="features_normalized_dense"):
    evaluator = AccuracyEvaluator(prediction_col="prediction_index", label_col="label")
    predictor = ModelPredictor(keras_model=model, features_col=features)
    transformer = LabelIndexTransformer(output_dim=10)
    test_set = test_set.select(features, "label")
    test_set = predictor.predict(test_set)
    test_set = transformer.transform(test_set)
    score = evaluator.evaluate(test_set)
    
    return score

### ADAG

In [ ]:
trainer = ADAG(keras_model=mlp, worker_optimizer=optimizer_mlp, loss=loss_mlp, num_workers=num_workers,
               batch_size=4, communication_window=5, num_epoch=1,
               features_col="features_normalized_dense", label_col="label_encoded")
# Modify the default parallelism factor.
trained_model = trainer.train(training_set)

In [19]:
# View the weights of the trained model.
trained_model.get_weights()

[array([[-0.02490237, -0.01861665,  0.03102627, ...,  0.01722135,
          0.02223415, -0.04933412],
        [-0.02634868,  0.03564246, -0.05392314, ..., -0.02999102,
         -0.01270337, -0.03888189],
        [ 0.00727941,  0.04553502, -0.01856072, ...,  0.0319587 ,
         -0.00354035, -0.03581727],
        ..., 
        [-0.03245988, -0.01220334,  0.019447  , ...,  0.05723321,
         -0.05618715, -0.0248918 ],
        [-0.02532675, -0.01772211,  0.05514754, ...,  0.03839124,
         -0.05036234, -0.03766601],
        [ 0.04610632,  0.01409597,  0.03790993, ..., -0.02038677,
         -0.03649681,  0.04742099]], dtype=float32),
 array([ -1.29682487e-02,   1.38744503e-01,  -3.10007334e-01,
         -3.04996595e-02,  -1.39434069e-01,  -4.05185074e-02,
         -2.09797233e-01,  -4.62490469e-01,  -6.72216356e-01,
         -1.83647368e-02,  -2.93090612e-01,   5.11649624e-02,
         -2.74094105e-01,  -9.03906003e-02,  -7.21242726e-01,
         -2.51375604e-02,  -1.40052319e-01,  -1

In [20]:
print("Training time: " + str(trainer.get_training_time()))
print("Accuracy: " + str(evaluate_accuracy(trained_model, test_set)))

Training time: 22619.2383449
Accuracy: 0.9859
